In [249]:
trades_df

,symbol,trade_id,exchange_id,price,size,trade_condition,trade_dt,arrive_dt,latency
0,SPY,52983527007092,8,327.450012,1,"[12, 37, 41]",2020-10-29 21:03:30.728,2020-10-29 21:03:30.853543,0 days 00:00:00.125543
1,SPY,71692756596565,4,327.399994,10,"[12, 37]",2020-10-29 21:03:34.190,2020-10-29 21:03:34.282885,0 days 00:00:00.092885
2,SPY,71692756596570,4,327.450012,1,"[12, 37]",2020-10-29 21:03:35.099,2020-10-29 21:03:35.138150,0 days 00:00:00.039150
3,SPY,52983575885875,11,327.399994,15,"[12, 37]",2020-10-29 21:03:39.846,2020-10-29 21:03:39.879815,0 days 00:00:00.033815
4,SPY,52983575885876,11,327.420013,99,"[14, 12, 37, 41]",2020-10-29 21:03:42.991,2020-10-29 21:03:43.027785,0 days 00:00:00.036785
...,...,...,...,...,...,...,...,...,...
5863,GLD,52983543183684,11,175.330002,35,"[14, 12, 37, 41]",2020-10-29 23:59:53.470,2020-10-29 23:59:53.588455,0 days 00:00:00.118455
5864,GLD,52983543185718,11,175.399994,0,"[38, 41]",2020-10-30 00:00:00.001,2020-10-30 00:00:00.259854,0 days 00:00:00.258854
5865,GLD,52983543185719,11,175.399994,837533,[15],2020-10-30 00:00:00.001,2020-10-30 00:00:00.259854,0 days 00:00:00.258854
5866,SPY,52983576548146,11,329.980011,0,"[38, 41]",2020-10-30 00:00:00.001,2020-10-30 00:00:00.261046,0 days 00:00:00.260046


In [ ]:
import ray
ray.shutdown()

In [208]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import pandas_bokeh
from qgrid import show_grid
# import finplot as fplt

import polygon_rest_api as papi
import polygon_streaming_api as pws
import polygon_backfill as pb
import polygon_s3 as ps3
import bar_samples as bs
import bar_labels as bl
import bar_meta as bm
import filters as ft
# import market_cluster as mc

import matplotlib as mpl
from matplotlib import style
style.use('fivethirtyeight')
mpl.rcParams['figure.figsize'] = [20, 7]

import ray
ray.init(ignore_reinit_error=True, num_cpus=4, dashboard_port=8080)

# from tqdm import tqdm
# import ipdb
# ipdb.set_trace(context=10)

# df.style.background_gradient(cmap='coolwarm', axis=None, vmin=-1, vmax=1)  # 'Blues', 'coolwarm', 'RdBu_r' & 'BrBG' are other good diverging colormaps
# pd.set_option('precision', 2)

# .plot_bokeh(sizing_mode="scale_height")

2020-10-29 22:52:00,463	ERROR worker.py:643 -- Calling ray.init() again after it has already been called.


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
symbol='GLD'
# symbol = 'market'
# date = '2020-01-02'
# start_date = '2020-01-01'
start_date = '2020-09-01'
end_date = '2020-10-01'
# symbols=['SPY', 'QQQ']
tick_type='trades'

result_path = '/Users/bobcolner/QuantClarity/pandas-polygon/data'

thresh = {
    # GT thresholds
    'renko_reveral_multiple': 2,
    'duration_sec': 60*60,
    # LT thresholds
    'min_duration_sec': 15,
    'min_tick_count': 30,
}

horizon_mins=30

In [ ]:
daily_bar_stats_df, labeled_bar_dates, stacked_bars_df = bm.bars_workflow_ray(
    result_path, 
    symbol, 
    start_date, 
    end_date, 
    thresh,
    imbalance_thresh=0.9
)

In [ ]:
daily_bar_stats_df.bar_count.describe()

In [ ]:
stacked_bars_df.bar_trigger.value_counts()

In [ ]:
stacked_bars_df.duration_min.hist(bins=100)

In [ ]:
daily_bar_stats_df.duration_min_mean.describe()

In [ ]:
bar_dates_df.price_wq50.diff().autocorr(1)

In [ ]:
# bar_dates_df.price_wmean.plot_bokeh(sizing_mode="scale_height")
bar_dates_df[200:400].price_wmean.plot()

In [ ]:
abs(bar_dates_df.label_rrr).value_counts()

In [ ]:
mh_dates_df = bar_dates_df.loc[bar_dates_df.close_at.dt.hour.between(13, 20)]

In [ ]:
mh_dates_df.label_outcome.value_counts()

In [ ]:
lbars_df[['price_wmean', 'label_rrr']].plot_bokeh(sizing_mode="scale_height")
# 17, 31

In [ ]:
lbars_df[['price_wmean', 'label_rrr']].plot(subplots=True, figsize=[17,12])